## 대신증권 API cybos plus 사용하기

### 필요 모듈 import

In [13]:
import win32com.client
import pandas as pd
import time
import datetime

### cybos plus 연결 체크

In [2]:
 # 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()
else:
    print("PLUS가 정상적 연결됨")


PLUS가 정상적 연결됨


### 삼성전자 현재 정보 불러오기

In [3]:
# 현재가 객체 구하기
objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
objStockMst.SetInputValue(0, 'A005930')   #종목 코드 - 삼성전자
objStockMst.BlockRequest()
 
# 현재가 통신 및 통신 에러 처리 
rqStatus = objStockMst.GetDibStatus()
rqRet = objStockMst.GetDibMsg1()

print("통신상태", rqStatus, rqRet)
if rqStatus != 0:
    exit()

# 현재가 정보 조회
code = objStockMst.GetHeaderValue(0)  #종목코드
name= objStockMst.GetHeaderValue(1)  # 종목명
time= objStockMst.GetHeaderValue(4)  # 시간
cprice= objStockMst.GetHeaderValue(11) # 종가
diff= objStockMst.GetHeaderValue(12)  # 대비
open= objStockMst.GetHeaderValue(13)  # 시가
high= objStockMst.GetHeaderValue(14)  # 고가
low= objStockMst.GetHeaderValue(15)   # 저가
offer = objStockMst.GetHeaderValue(16)  #매도호가
bid = objStockMst.GetHeaderValue(17)   #매수호가
vol= objStockMst.GetHeaderValue(18)   #거래량
vol_value= objStockMst.GetHeaderValue(19)  #거래대금
 
# 예상 체결관련 정보
exFlag = objStockMst.GetHeaderValue(58) #예상체결가 구분 플래그
exPrice = objStockMst.GetHeaderValue(55) #예상체결가
exDiff = objStockMst.GetHeaderValue(56) #예상체결가 전일대비
exVol = objStockMst.GetHeaderValue(57) #예상체결수량
 
 
print("코드", code)
print("이름", name)
print("시간", time)
print("종가", cprice)
print("대비", diff)
print("시가", open)
print("고가", high)
print("저가", low)
print("매도호가", offer)
print("매수호가", bid)
print("거래량", vol)
print("거래대금", vol_value)
 
 
if (exFlag == ord('0')):
    print("장 구분값: 동시호가와 장중 이외의 시간")
elif (exFlag == ord('1')) :
    print("장 구분값: 동시호가 시간")
elif (exFlag == ord('2')):
    print("장 구분값: 장중 또는 장종료")
 
print("예상체결가 대비 수량")
print("예상체결가", exPrice)
print("예상체결가 대비", exDiff)
print("예상체결수량", exVol)

통신상태 0 0027 조회가 완료되었습니다.(stock.new.mst)
코드 A005930
이름 삼성전자
시간 1559
종가 75600
대비 2300
시가 73900
고가 75700
저가 73900
매도호가 75600
매수호가 75500
거래량 20803852
거래대금 155834891
장 구분값: 장중 또는 장종료
예상체결가 대비 수량
예상체결가 75600
예상체결가 대비 2300
예상체결수량 1534422


### 코스피/코스닥 전 종목 코드 가져와 저장하기

In [4]:
rows = list()
CPE_MARKET_KIND = {'KOSPI':1, 'KOSDAQ':2}
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
for key, value in CPE_MARKET_KIND.items():
    codeList = instCpCodeMgr.GetStockListByMarket(value)
    for code in codeList:
        name = instCpCodeMgr.CodeToName(code)
        sectionKind = instCpCodeMgr.GetStockSectionKind(code)
        group = instCpCodeMgr.GetStockIndustryCode(code)
        row = [code, name, key, sectionKind, group]
        rows.append(row)

print('모든 종목을 불러왔습니다')
stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind', 'group_category'])
stockitems.loc[stockitems['sectionKind'] == 10, 'section'] = 'ETF'
stockitems.to_csv('./data/stockitems.csv', index=False)
print('파일을 저장하였습니다.')

모든 종목을 불러왔습니다
파일을 저장하였습니다.


In [5]:
stockitems

,code,name,section,sectionKind,group_category
0,A000020,동화약품,KOSPI,1,009
1,A000040,KR모터스,KOSPI,1,015
2,A000050,경방,KOSPI,1,016
3,A000060,메리츠화재,KOSPI,1,025
4,A000070,삼양홀딩스,KOSPI,1,021
...,...,...,...,...,...
3139,A950170,JTC,KOSDAQ,6,647
3140,A950180,SNK,KOSDAQ,6,658
3141,A950190,미투젠,KOSDAQ,6,658
3142,A950200,소마젠,KOSDAQ,6,673


### 코스피/코스닥 주가 데이터 가져오기 
-  종목코드, 종목명, 섹션, 날짜,  시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액, 외국인 보유량, 외국인 보유비율, 기관순매수, 기관누적순매수
   ```py
   column_dailychart = ['code', 'name', 'section', 'date', 'open', 'high', 'low', 'close', 'vol','value', 'n_stock', 'agg_price', 'foreign_vol' ,'foreign_rate','agency_buy', 'agency_netbuy']
   ```

#### 삼성전자는 어떤 정보를 가졌을까?

In [79]:
stockitems = pd.read_csv("./data/stockitems.csv")
stockitems[stockitems["name"] == "삼성전자"]

,code,name,section,sectionKind,group_category
303,A005930,삼성전자,KOSPI,1,13


#### 전 종목에 해당하는 2020년도 주가 데이터 가져오기

In [46]:
# 날짜, 종목명, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액, 외국인 보유비율, 기관순매수, 기관누적순매수
column_dailychart = ['code', 'name', 'section', 'date', 'open', 'high', 'low', 'close', 'vol', 'value', 
                     'n_stock', 'agg_price', 'foreign_vol', 'foreign_rate','agency_buy', 'agency_netbuy']

stockitems = pd.read_csv('./data/stockitems.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, '20201231')
# instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, '20200101')
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

for idx, stockitem in stockitems.iterrows():
    remain_request_count = nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count = nCpCybos.GetLimitRemainCount(1)
            
            if remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, stockitem['code'])
    
    # BlockRequest
    instStockChart.BlockRequest()
    
    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)
    
    # GetDataValue
    for i in range(numData):
        row[0] = stockitem['code']
        row[1] = stockitem['name']
        row[2] = stockitem['section']# 코스피, 코스닥, ETF 여부
        row[3] = instStockChart.GetDataValue(0, i)# 날짜
        row[4] = instStockChart.GetDataValue(1, i) # 시가
        row[5] = instStockChart.GetDataValue(2, i) # 고가
        row[6] = instStockChart.GetDataValue(3, i) # 저가
        row[7] = instStockChart.GetDataValue(4, i) # 종가
        row[8] = instStockChart.GetDataValue(5, i) # 거래량
        row[9] = instStockChart.GetDataValue(6, i) # 거래대금
        row[10] = instStockChart.GetDataValue(7, i) # 상장주식수
        row[11] = instStockChart.GetDataValue(8, i) # 시가총액
        row[12] = instStockChart.GetDataValue(9, i) # 외국인 보유량
        row[13] = instStockChart.GetDataValue(10, i) # 외국인 보율비율
        row[14] = instStockChart.GetDataValue(11, i) # 기관순매수
        row[15] = instStockChart.GetDataValue(12, i) # 기관누적순매수
        rows.append(list(row))
        
print('데이터를 모두 불러왔습니다.')

A000020 동화약품 남은 요청 :  50
A000040 KR모터스 남은 요청 :  49
A000050 경방 남은 요청 :  48
A000060 메리츠화재 남은 요청 :  47
A000070 삼양홀딩스 남은 요청 :  46
A000075 삼양홀딩스우 남은 요청 :  45
A000080 하이트진로 남은 요청 :  44
A000087 하이트진로2우B 남은 요청 :  43
A000100 유한양행 남은 요청 :  42
A000105 유한양행우 남은 요청 :  41
A000120 CJ대한통운 남은 요청 :  40
A000140 하이트진로홀딩스 남은 요청 :  39
A000145 하이트진로홀딩스우 남은 요청 :  38
A000150 두산 남은 요청 :  37
A000155 두산우 남은 요청 :  36
A000157 두산2우B 남은 요청 :  35
A000180 성창기업지주 남은 요청 :  34
A000210 DL 남은 요청 :  33
A000215 DL우 남은 요청 :  32
A000220 유유제약 남은 요청 :  31
A000225 유유제약1우 남은 요청 :  30
A000227 유유제약2우B 남은 요청 :  29
A000230 일동홀딩스 남은 요청 :  28
A000240 한국앤컴퍼니 남은 요청 :  27
A000270 기아 남은 요청 :  26
A000300 대유플러스 남은 요청 :  25
A000320 노루홀딩스 남은 요청 :  24
A000325 노루홀딩스우 남은 요청 :  23
A000370 한화손해보험 남은 요청 :  22
A000390 삼화페인트 남은 요청 :  21
A000400 롯데손해보험 남은 요청 :  20
A000430 대원강업 남은 요청 :  19
A000480 조선내화 남은 요청 :  59
A000490 대동 남은 요청 :  58
A000500 가온전선 남은 요청 :  57
A000520 삼일제약 남은 요청 :  56
A000540 흥국화재 남은 요청 :  55
A000545 흥국화재우 남은 요청 :  54
A000547 흥국화재2우B 남은 

A005960 동부건설 남은 요청 :  23
A005965 동부건설우 남은 요청 :  22
A006040 동원산업 남은 요청 :  21
A006060 화승인더 남은 요청 :  20
A006090 사조오양 남은 요청 :  19
A006110 삼아알미늄 남은 요청 :  18
A006120 SK디스커버리 남은 요청 :  17
A006125 SK디스커버리우 남은 요청 :  16
A006200 한국전자홀딩스 남은 요청 :  15
A006220 제주은행 남은 요청 :  14
A006260 LS 남은 요청 :  13
A006280 녹십자 남은 요청 :  12
A006340 대원전선 남은 요청 :  11
A006345 대원전선우 남은 요청 :  10
A006360 GS건설 남은 요청 :  9
A006370 대구백화점 남은 요청 :  8
A006380 카프로 남은 요청 :  7
A006390 한일현대시멘트 남은 요청 :  6
A006400 삼성SDI 남은 요청 :  5
A006405 삼성SDI우 남은 요청 :  4
A006490 인스코비 남은 요청 :  3
A006570 대림통상 남은 요청 :  2
A006650 대한유화 남은 요청 :  1
A006660 삼성공조 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A006740 영풍제지 남은 요청 :  59
A006800 미래에셋증권 남은 요청 :  58
A006805 미래에셋증권우 남은 요청 :  57
A00680K 미래에셋증권2우B 남은 요청 :  56
A006840 AK홀딩스 남은 요청 :  55
A006880 신송홀딩스 남은 요청 :  54
A006890 태경케미컬 남은 요청 :  53
A006980 우성사료 남은 요청 :  52
A007070 GS리테일 남은 요청 :  51
A007110 일신석재 남은 요청 :  50
A007120 미래아이앤지 남은 요청 :  49
A007160 사조

A034300 신세계건설 남은 요청 :  17
A034310 NICE 남은 요청 :  16
A034590 인천도시가스 남은 요청 :  15
A034730 SK 남은 요청 :  14
A03473K SK우 남은 요청 :  13
A034830 한국토지신탁 남은 요청 :  12
A035000 지투알 남은 요청 :  11
A035150 백산 남은 요청 :  10
A035250 강원랜드 남은 요청 :  9
A035420 NAVER 남은 요청 :  8
A035510 신세계 I&C 남은 요청 :  7
A035720 카카오 남은 요청 :  6
A036420 제이콘텐트리 남은 요청 :  5
A036460 한국가스공사 남은 요청 :  4
A036530 SNT홀딩스 남은 요청 :  3
A036570 엔씨소프트 남은 요청 :  2
A036580 팜스코 남은 요청 :  1
A037270 YG PLUS 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A037560 LG헬로비전 남은 요청 :  59
A037710 광주신세계 남은 요청 :  58
A039130 하나투어 남은 요청 :  57
A039490 키움증권 남은 요청 :  56
A039570 HDC아이콘트롤스 남은 요청 :  55
A041650 상신브레이크 남은 요청 :  54
A042660 대우조선해양 남은 요청 :  53
A042670 두산인프라코어 남은 요청 :  52
A042700 한미반도체 남은 요청 :  51
A044380 주연테크 남은 요청 :  50
A044450 KSS해운 남은 요청 :  49
A044820 코스맥스비티아이 남은 요청 :  48
A047040 대우건설 남은 요청 :  47
A047050 포스코인터내셔널 남은 요청 :  46
A047400 유니온머티리얼 남은 요청 :  45
A047810 한국항공우주 남은 요청 :  44
A049770 동원F&B 남은 요청 :  43


A152380 KODEX 국채선물10년 남은 요청 :  36
A152500 KINDEX 레버리지 남은 요청 :  35
A152550 한국ANKOR유전 남은 요청 :  34
A152870 파워 200 남은 요청 :  33
A153130 KODEX 단기채권 남은 요청 :  32
A153270 KOSEF 코스피100 남은 요청 :  31
A153360 하이골드3호 남은 요청 :  30
A155660 DSR 남은 요청 :  29
A155900 바다로19호 남은 요청 :  28
A156080 KODEX MSCI Korea 남은 요청 :  27
A157450 TIGER 단기통안채 남은 요청 :  26
A157490 TIGER 소프트웨어 남은 요청 :  25
A157500 TIGER 증권 남은 요청 :  24
A159800 마이티 코스피100 남은 요청 :  23
A160580 TIGER 구리실물 남은 요청 :  22
A161000 애경유화 남은 요청 :  21
A161390 한국타이어앤테크놀로지 남은 요청 :  20
A161510 ARIRANG 고배당주 남은 요청 :  19
A161890 한국콜마 남은 요청 :  18
A163560 동일고무벨트 남은 요청 :  17
A166400 TIGER 200커버드콜5%OTM 남은 요청 :  16
A167860 KOSEF 국고채10년레버리지 남은 요청 :  15
A168300 KTOP 코스피50 남은 요청 :  14
A168490 한국패러랠 남은 요청 :  13
A168580 KINDEX 중국본토CSI300 남은 요청 :  12
A169950 KODEX 중국본토 A50 남은 요청 :  11
A170900 동아에스티 남은 요청 :  10
A172580 하이골드12호 남은 요청 :  9
A174350 TIGER 로우볼 남은 요청 :  8
A174360 KBSTAR 중국본토대형주CSI100 남은 요청 :  7
A175330 JB금융지주 남은 요청 :  6
A176710 파워 중기국고채 남은 요청 :  5
A176950 KODEX 국채선물1

A275280 KODEX MSCI모멘텀 남은 요청 :  43
A275290 KODEX MSCI밸류 남은 요청 :  42
A275300 KODEX MSCI퀄리티 남은 요청 :  41
A275750 KBSTAR 코스닥150선물인버스 남은 요청 :  40
A275980 TIGER 글로벌4차산업혁신기술(합성 H) 남은 요청 :  39
A276000 TIGER 글로벌자원생산기업(합성 H) 남은 요청 :  38
A276650 KBSTAR 글로벌4차산업IT(합성 H) 남은 요청 :  37
A276970 KODEX 미국S&P고배당커버드콜(합성 H) 남은 요청 :  36
A276990 KODEX 글로벌4차산업로보틱스(합성) 남은 요청 :  35
A277540 KINDEX S&P아시아TOP50 남은 요청 :  34
A277630 TIGER 코스피 남은 요청 :  33
A277640 TIGER 코스피대형주 남은 요청 :  32
A277650 TIGER 코스피중형주 남은 요청 :  31
A278240 KBSTAR 코스닥150선물레버리지 남은 요청 :  30
A278420 ARIRANG ESG우수기업 남은 요청 :  29
A278530 KODEX 200TR 남은 요청 :  28
A278540 KODEX MSCI Korea TR 남은 요청 :  27
A278620 ARIRANG 단기채권액티브 남은 요청 :  26
A279530 KODEX 고배당 남은 요청 :  25
A279540 KODEX 최소변동성 남은 요청 :  24
A280320 KINDEX 미국IT인터넷S&P(합성 H) 남은 요청 :  23
A280360 롯데제과 남은 요청 :  22
A280920 ARIRANG 주도업종 남은 요청 :  21
A280930 KODEX 미국러셀2000(H) 남은 요청 :  20
A280940 KODEX 골드선물인버스(H) 남은 요청 :  19
A281820 케이씨텍 남은 요청 :  18
A281990 KBSTAR 중소형고배당 남은 요청 :  17
A282000 KBSTAR 국고채3년선물인버스 남

...

Q550041 QV 미국 IT TOP5 ETN(H) 남은 요청 :  31
Q550042 QV 레버리지 WTI원유 선물 ETN(H) 남은 요청 :  30
Q550043 QV 인버스 레버리지 WTI원유 선물 ETN(H) 남은 요청 :  29
Q550044 QV 레버리지 S&P500 ETN 남은 요청 :  28
Q550045 QV 인버스 레버리지 S&P500 ETN 남은 요청 :  27
Q550057 QV 코스피 변동성 매칭형 양매도 ETN 남은 요청 :  26
Q550059 QV iSelect 글로벌 EMP ETN(H) 남은 요청 :  25
Q550060 QV S&P500 VIX S/T 선물 ETN C 남은 요청 :  24
Q550061 QV 인버스 미국 IT TOP5 ETN(H) 남은 요청 :  23
Q550062 QV 레버리지 금 선물 ETN(H) 남은 요청 :  22
Q550063 QV 인버스 레버리지 금 선물 ETN(H) 남은 요청 :  21
Q570019 TRUE 코스피 양매도 5% OTM ETN 남은 요청 :  20
Q570020 TRUE 코스피 커버드콜 ETN B 남은 요청 :  19
Q570021 TRUE 코스피 커버드풋 ETN B 남은 요청 :  18
Q570022 TRUE 레버리지 S&P500 선물 ETN(H) 남은 요청 :  17
Q570023 TRUE 인버스 2X S&P500 선물 ETN(H) 남은 요청 :  16
Q570025 TRUE 인버스 유로스탁스50 ETN(H) B 남은 요청 :  15
Q570027 TRUE 인버스 2X 유로스탁스50 ETN(H) 남은 요청 :  14
Q570028 TRUE 인버스 HSCEI ETN(H) B 남은 요청 :  13
Q570029 TRUE 코스피 양매도 3% OTM ETN 남은 요청 :  12
Q570030 TRUE 코스피 양매도 ATM ETN 남은 요청 :  11
Q570031 TRUE 레버리지 HSCEI ETN(H) 남은 요청 :  10
Q570032 TRUE 인버스 2X HSCEI ETN(H) 남은

A032960 동일기연 남은 요청 :  9
A032980 바이온 남은 요청 :  8
A033050 제이엠아이 남은 요청 :  7
A033100 제룡전기 남은 요청 :  6
A033110 코너스톤네트웍스 남은 요청 :  5
A033130 디지틀조선 남은 요청 :  4
A033160 엠케이전자 남은 요청 :  3
A033170 시그네틱스 남은 요청 :  2
A033200 모아텍 남은 요청 :  1
A033230 인성정보 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A033290 코웰패션 남은 요청 :  59
A033310 엠투엔 남은 요청 :  58
A033320 제이씨현시스템 남은 요청 :  57
A033340 좋은사람들 남은 요청 :  56
A033430 디에스티 남은 요청 :  55
A033500 동성화인텍 남은 요청 :  54
A033540 파라텍 남은 요청 :  53
A033560 블루콤 남은 요청 :  52
A033640 네패스 남은 요청 :  51
A033790 스카이문스테크놀로지 남은 요청 :  50
A033830 티비씨 남은 요청 :  49
A034230 파라다이스 남은 요청 :  48
A034810 해성산업 남은 요청 :  47
A034940 조아제약 남은 요청 :  46
A034950 한국기업평가 남은 요청 :  45
A035080 인터파크 남은 요청 :  44
A035200 프럼파스트 남은 요청 :  43
A035290 골드앤에스 남은 요청 :  42
A035460 기산텔레콤 남은 요청 :  41
A035600 KG이니시스 남은 요청 :  40
A035610 솔본 남은 요청 :  39
A035620 바른손이앤에이 남은 요청 :  38
A035760 CJ ENM 남은 요청 :  37
A035810 이지홀딩스 남은 요청 :  36
A035890 서희건설 남은 요청 :  35
A035900 JYP Ent. 남

A059100 아이컴포넌트 남은 요청 :  4
A059120 아진엑스텍 남은 요청 :  3
A059210 메타바이오메드 남은 요청 :  2
A059270 해성티피씨 남은 요청 :  1
A060150 인선이엔티 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A060230 이그잭스 남은 요청 :  59
A060240 룽투코리아 남은 요청 :  58
A060250 NHN한국사이버결제 남은 요청 :  57
A060260 뉴보텍 남은 요청 :  56
A060280 큐렉소 남은 요청 :  55
A060300 레드로버 남은 요청 :  54
A060310 3S 남은 요청 :  53
A060370 KT서브마린 남은 요청 :  52
A060380 동양에스텍 남은 요청 :  51
A060480 국일신동 남은 요청 :  50
A060540 에스에이티 남은 요청 :  49
A060560 홈센타홀딩스 남은 요청 :  48
A060570 드림어스컴퍼니 남은 요청 :  47
A060590 씨티씨바이오 남은 요청 :  46
A060720 KH바텍 남은 요청 :  45
A060850 영림원소프트랩 남은 요청 :  44
A060900 대한그린파워 남은 요청 :  43
A061040 알에프텍 남은 요청 :  42
A061250 화일약품 남은 요청 :  41
A061970 엘비세미콘 남은 요청 :  40
A062860 티엘아이 남은 요청 :  39
A062970 피피아이 남은 요청 :  38
A063080 게임빌 남은 요청 :  37
A063170 서울옥션 남은 요청 :  36
A063440 SM Life Design 남은 요청 :  35
A063570 한국전자금융 남은 요청 :  34
A063760 이엘피 남은 요청 :  33
A064090 에프앤리퍼블릭 남은 요청 :  32
A064240 홈캐스트 남은 요청 :  31
A064260 다날 남은 요청 :  3

대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A208710 바이오로그디바이스 남은 요청 :  59
A208860 엔지스테크널러지 남은 요청 :  58
A211270 AP위성 남은 요청 :  57
A212560 네오오토 남은 요청 :  56
A213090 미래테크놀로지 남은 요청 :  55
A213420 덕산네오룩스 남은 요청 :  54
A214150 클래시스 남은 요청 :  53
A214180 민앤지 남은 요청 :  52
A214260 라파스 남은 요청 :  51
A214270 퓨쳐스트림네트웍스 남은 요청 :  50
A214310 에스엘바이오닉스 남은 요청 :  49
A214370 케어젠 남은 요청 :  48
A214430 아이쓰리시스템 남은 요청 :  47
A214450 파마리서치 남은 요청 :  46
A214610 미코바이오메드 남은 요청 :  45
A214680 디알텍 남은 요청 :  44
A214870 뉴지랩파마 남은 요청 :  43
A215000 골프존 남은 요청 :  42
A215090 휴센텍 남은 요청 :  41
A215100 로보로보 남은 요청 :  40
A215200 메가스터디교육 남은 요청 :  39
A215360 우리산업 남은 요청 :  38
A215380 우정바이오 남은 요청 :  37
A215480 토박스코리아 남은 요청 :  36
A215600 신라젠 남은 요청 :  35
A215790 이노인스트루먼트 남은 요청 :  34
A216050 인크로스 남은 요청 :  33
A216080 제테마 남은 요청 :  32
A217190 제너셈 남은 요청 :  31
A217270 넵튠 남은 요청 :  30
A217330 싸이토젠 남은 요청 :  29
A217480 에스디생명공학 남은 요청 :  28
A217500 러셀 남은 요청 :  27
A217600 켐온 남은 요청 :  26
A217620 디딤 남은 요청 :  25
A217730 강스템바이오텍 남은 요청 :  24
A21782

대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
A330860 네패스아크 남은 요청 :  59
A330990 케이비제19호스팩 남은 요청 :  58
A331380 유진스팩5호 남은 요청 :  57
A331520 교보9호스팩 남은 요청 :  56
A331920 셀레믹스 남은 요청 :  55
A332290 대신밸런스제7호스팩 남은 요청 :  54
A332370 아이디피 남은 요청 :  53
A332570 와이팜 남은 요청 :  52
A332710 하나금융14호스팩 남은 요청 :  51
A333050 신한제6호스팩 남은 요청 :  50
A333430 일승 남은 요청 :  49
A333620 엔시스 남은 요청 :  48
A334970 프레스티지바이오로직스 남은 요청 :  47
A335810 프리시젼바이오 남은 요청 :  46
A335870 IBKS제12호스팩 남은 요청 :  45
A335890 비올 남은 요청 :  44
A336060 유안타제5호스팩 남은 요청 :  43
A336570 대신밸런스제8호스팩 남은 요청 :  42
A337450 SK5호스팩 남은 요청 :  41
A337930 브랜드엑스코퍼레이션 남은 요청 :  40
A338220 뷰노 남은 요청 :  39
A339950 아이비김영 남은 요청 :  38
A340120 하이제5호스팩 남은 요청 :  37
A340350 SK6호스팩 남은 요청 :  36
A340360 다보링크 남은 요청 :  35
A340440 한화플러스제1호스팩 남은 요청 :  34
A340570 티앤엘 남은 요청 :  33
A340930 유일에너테크 남은 요청 :  32
A341160 하나금융15호스팩 남은 요청 :  31
A342550 케이비제20호스팩 남은 요청 :  30
A343510 하나금융16호스팩 남은 요청 :  29
A344050 신영스팩6호 남은 요청 :  28
A347000 센코 남은 요청 :  27
A347140 케이프이에스제4호 남은 요청 :  26
A347700 라이프시맨틱스 남은 

In [47]:
all_stock_dailyChart_2020 = pd.DataFrame(rows, columns=column_dailychart)
all_stock_dailyChart_2020 = all_stock_dailyChart_2020.sort_values(["code", "date"])
all_stock_dailyChart_2020 = all_stock_dailyChart_2020.reset_index(drop=True)
all_stock_dailyChart_2020.to_csv("./data/all_stock_daily_chart_2020.csv", index=False)
all_stock_dailyChart_2020

,code,name,section,date,open,high,low,close,vol,value,n_stock,agg_price,foreign_rate,agency_buy,agency_netbuy
0,A000020,동화약품,KOSPI,20200102,8340,8400,8290,8400,111305,931000000,27931000,234620000000,6.05,-3697,49729
1,A000020,동화약품,KOSPI,20200103,8400,8450,8290,8360,96437,805000000,27931000,233503000000,6.09,3226,52955
2,A000020,동화약품,KOSPI,20200106,8290,8330,8120,8180,73230,599000000,27931000,228475000000,6.08,-1741,51214
3,A000020,동화약품,KOSPI,20200107,8200,8280,8090,8160,117904,961000000,27931000,227916000000,6.13,1457,52671
4,A000020,동화약품,KOSPI,20200108,8170,8170,7830,7930,263246,2095000000,27931000,221492000000,6.12,-1078,51593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709580,Q700002,하나 KRX300 ETN,KOSPI,20201223,9950,9980,9865,9980,6011,60000000,1000000,9980000000,0.00,9,-2790
709581,Q700002,하나 KRX300 ETN,KOSPI,20201224,10075,10210,10075,10110,8855,89000000,1000000,10110000000,0.00,113,-2677
709582,Q700002,하나 KRX300 ETN,KOSPI,20201228,10300,10300,10195,10195,2310,24000000,1000000,10195000000,0.00,310,-2367
709583,Q700002,하나 KRX300 ETN,KOSPI,20201229,10230,10230,10190,10190,423,4000000,1000000,10190000000,0.00,423,-1944


#### 함수화 하기
- 입력: **stockitems, start, end**
    - stockitems: 
        - type: DataFrame
        - 주식 데이터를 얻고자 하는 종목의 정보
            - stockitems: ["code", "name", "section"]
    - start/end: 
        - type: string (형식: YYYYMMDD)
        - 얻고자 하는 시작 날짜/끝 날짜
- 출력: **stockitems_dailychart**
    - type: DataFrame
    - start ~ end에 해당하는 주식 데이터
    - return 하기 전에 각 종목별 주식데이터와 csv 파일로 저장 (경로: "./data/": 경로 못찾으면 에러)
    

In [83]:
def get_stock_data(stockitems, start, end):
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    
    # 날짜, 종목명, 시가, 고가, 저가, 종가, 거래량, 거래대금, 
    # 상장주식수, 시가총액, 외국인 보유수량, 외국인 보유비율, 기관순매수, 기관누적순매수
    column_dailychart = ['code', 'name', 'section', 'date', 'open', 'high', 'low', 'close', 'vol', 'value',
                         'n_stock', 'agg_price', 'foreign_vol', 'foreign_rate','agency_buy', 'agency_netbuy']
    
    stockitems_dailychart = pd.DataFrame()
    
    s_year, e_year = start[:4], end[:4]
    s_day, e_day = start[4:], end[4:]
    
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21))
    instStockChart.SetInputValue(6, ord('D'))
    instStockChart.SetInputValue(9, ord('1'))

    for idx, stockitem in stockitems.iterrows():
        dailychart = pd.DataFrame()
        row = list(range(len(column_dailychart)))
        rows = list()
        
        instStockChart.SetInputValue(0, stockitem['code'])
        print(stockitem['name'], "주식 데이터를 가져옵니다.")
        
        y = int(s_year)
        while y <= int(e_year):
            # 시작일, 종료일 계산
            if s_year == y:
                s_y, s_d = s_year, s_day
            else:
                s_y, s_d = str(y), '0101'
                
            if e_year == y:
                e_y, e_d = e_year, e_day
            else:
                e_y, e_d = str(y), '1231'
                
            print(s_y + s_d + '~' + e_y + e_d + '의 데이터를 가져옵니다.')
            instStockChart.SetInputValue(3, s_y + s_d)
            instStockChart.SetInputValue(2, e_y + e_d)
            
            # 남은요청 있는지 확인
            remain_request_count = nCpCybos.GetLimitRemainCount(1)
            print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)

            if remain_request_count == 0:
                print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

                while True:
                    time.sleep(2)
                    remain_request_count = nCpCybos.GetLimitRemainCount(1)

                    if remain_request_count > 0:
                        print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                        break
                    print('대기 중...')
                    
            # BlockRequest
            instStockChart.BlockRequest()

            # GetHeaderValue
            numData = instStockChart.GetHeaderValue(3)
            numField = instStockChart.GetHeaderValue(1)
            # GetDataValue
            print(numData)
            for i in range(numData):
                row[0] = stockitem['code']
                row[1] = stockitem['name']
                row[2] = stockitem['section']# 코스피, 코스닥, ETF 여부
                row[3] = instStockChart.GetDataValue(0, i)# 날짜
                row[4] = instStockChart.GetDataValue(1, i) # 시가
                row[5] = instStockChart.GetDataValue(2, i) # 고가
                row[6] = instStockChart.GetDataValue(3, i) # 저가
                row[7] = instStockChart.GetDataValue(4, i) # 종가
                row[8] = instStockChart.GetDataValue(5, i) # 거래량
                row[9] = instStockChart.GetDataValue(6, i) # 거래대금
                row[10] = instStockChart.GetDataValue(7, i) # 상장주식수
                row[11] = instStockChart.GetDataValue(8, i) # 시가총액
                row[12] = instStockChart.GetDataValue(9, i) # 외국인 현 보유수량
                row[13] = instStockChart.GetDataValue(10, i) # 외국인 보율비율
                row[14] = instStockChart.GetDataValue(11, i) # 기관순매수
                row[15] = instStockChart.GetDataValue(12, i) # 기관누적순매수
                rows.append(list(row))
            df =  pd.DataFrame(data = rows, columns= column_dailychart)
            dailychart = pd.concat([dailychart, df])
            y += 1    
        dailychart = dailychart.sort_values("date")
        dailychart = dailychart.drop_duplicates(['date'])
        dailychart = dailychart.reset_index(drop=True)
#         dailychart.to_csv("./data/" + stockitem['name'] + "-stock-data-" 
#                           + str(dailychart['date'][0])[:4]
#                           + "-" + e_year + ".csv", index=False)
        stockitems_dailychart = pd.concat([stockitems_dailychart, dailychart])
    return stockitems_dailychart

#### 삼성, 카카오, 셀트리온 2000-2021년 주가 데이터 뽑기

In [31]:
stockitems = pd.DataFrame(data=[["A005930", "samsung", "KOSPI"],
                               ["A035720", "kakao", "KOSPI"], 
                               ["A068270", "celltrion", "KOSPI"]], columns=["code", "name", "section"])
stockitems

,code,name,section
0,A005930,samsung,KOSPI
1,A035720,kakao,KOSPI
2,A068270,celltrion,KOSPI


In [32]:
stockitems_dailychart = get_stock_data(stockitems, '20000101', '20211231')

samsung 주식 데이터를 가져옵니다.
20000101~20001231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  60
241
20010101~20011231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  59
246
20020101~20021231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  58
244
20030101~20031231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  57
247
20040101~20041231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  56
249
20050101~20051231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  55
249
20060101~20061231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  54
247
20070101~20071231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  53
246
20080101~20081231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  52
248
20090101~20091231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  51
253
20100101~20101231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  50
251
20110101~20111231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  49
248
20120101~20121231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  48
248
20130101~20131231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  47
247
20140101~20141231의 데이터를 가져옵니다.
A005930 samsung 남은 요청 :  46
245
20150101~20151231의 데이터를 가져옵니다.
A

In [33]:
stockitems_dailychart

,code,name,section,date,open,high,low,close,vol,value,n_stock,agg_price,foreign_vol,foreign_rate,agency_buy,agency_netbuy
0,A005930,samsung,KOSPI,20000104,6000,6110,5660,6110,74198350,444233000000,7491938000,45775741000000,3542821,47.290001,0,0
1,A005930,samsung,KOSPI,20000105,5800,6060,5520,5580,74680200,433783000000,7491938000,41805014000000,3545424,47.320000,0,0
2,A005930,samsung,KOSPI,20000106,5750,5780,5580,5620,54390500,308005000000,7491938000,42104691000000,3572088,47.680000,0,0
3,A005930,samsung,KOSPI,20000107,5560,5670,5360,5540,40309750,222718000000,7491938000,41505336000000,3574158,47.709999,0,0
4,A005930,samsung,KOSPI,20000110,5600,5770,5580,5770,46880750,265754000000,7491938000,43228482000000,3584616,47.849998,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,A068270,celltrion,KOSPI,20210818,275000,277000,270000,274000,420737,115373000000,137934000,37793916000000,28604910,20.740000,45049,8706957
3980,A068270,celltrion,KOSPI,20210819,271500,279500,270000,271000,500781,137259000000,137934000,37380114000000,28651573,20.770000,48378,8755335
3981,A068270,celltrion,KOSPI,20210820,273500,276500,265000,266000,647705,175016000000,137934000,36690444000000,28528411,20.680000,53388,8808723
3982,A068270,celltrion,KOSPI,20210823,269000,286500,269000,284000,1355766,382027000000,137934000,39173256000000,28871601,20.930000,235785,9044508


#### 대한항공 1980-2021년 주가 데이터 뽑기

In [34]:
stockitems = pd.DataFrame(data=[["A003490", "koreanAir", "KOSPI"]], columns=["code", "name", "section"])
stockitems

,code,name,section
0,A003490,koreanAir,KOSPI


In [35]:
stockitems_dailychart = get_stock_data(stockitems, '19800101', '20211231')

koreanAir 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  60
291
19810101~19811231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  59
291
19820101~19821231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  58
296
19830101~19831231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  57
295
19840101~19841231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  56
293
19850101~19851231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  55
294
19860101~19861231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  54
292
19870101~19871231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  53
292
19880101~19881231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  52
293
19890101~19891231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  51
289
19900101~19901231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  50
291
19910101~19911231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  49
292
19920101~19921231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  48
294
19930101~19931231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  47
296
19940101~19941231의 데이터를 가져옵니다.
A003490 koreanAir 남은 요청 :  46
296


#### WTI원유선물(H) 시작일(상장일)-2021년 주가 데이터 뽑기

In [36]:
stockitems = pd.DataFrame(data=[["A261220", "WTI원유선물(H)", "KODEX"]], columns=["code", "name", "section"])
stockitems

,code,name,section
0,A261220,WTI원유선물(H),KODEX


In [37]:
stockitems_dailychart = get_stock_data(stockitems, '19800101', '20211231')

WTI원유선물(H) 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  60
0
19810101~19811231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  59
0
19820101~19821231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  58
0
19830101~19831231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  57
0
19840101~19841231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  56
0
19850101~19851231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  55
0
19860101~19861231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  54
0
19870101~19871231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  53
0
19880101~19881231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  52
0
19890101~19891231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  51
0
19900101~19901231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  50
0
19910101~19911231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  49
0
19920101~19921231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  48
0
19930101~19931231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  47
0
19940101~19941231의 데이터를 가져옵니다.
A261220 WTI원유선물(H) 남은 요청 :  46
0
19950101~19951

In [38]:
stockitems_dailychart

,code,name,section,date,open,high,low,close,vol,value,n_stock,agg_price,foreign_vol,foreign_rate,agency_buy,agency_netbuy
0,A261220,WTI원유선물(H),KODEX,20161227,20105,20105,20025,20055,50225,1007000000,750000,15041000000,0,0.00,-25,-25
1,A261220,WTI원유선물(H),KODEX,20161228,20315,20330,20285,20285,151126,3069000000,750000,15213000000,0,0.00,-1879,-1904
2,A261220,WTI원유선물(H),KODEX,20161229,20250,20315,20250,20300,141880,2875000000,750000,15225000000,0,0.00,-3,-1907
3,A261220,WTI원유선물(H),KODEX,20170102,20395,20395,20345,20345,116034,2366000000,750000,15258000000,0,0.00,-2537,-4444
4,A261220,WTI원유선물(H),KODEX,20170103,20350,20430,20350,20360,136971,2793000000,750000,15270000000,0,0.00,-1008,-5452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,A261220,WTI원유선물(H),KODEX,20210818,10090,10160,10085,10150,303166,3068000000,21500000,218225000000,5000,0.02,-127016,-16575519
1141,A261220,WTI원유선물(H),KODEX,20210819,9835,9835,9730,9735,1157749,11323000000,21100000,205408000000,5000,0.02,-51718,-16627237
1142,A261220,WTI원유선물(H),KODEX,20210820,9705,9750,9670,9715,357942,3477000000,21125000,205229000000,5000,0.02,-25430,-16652667
1143,A261220,WTI원유선물(H),KODEX,20210823,9495,9665,9495,9655,412256,3945000000,21300000,205651000000,5000,0.02,5109,-16647558


### 그룹 카테고리(테마)는 어떤 것이 있는지 확인

In [60]:
group_list = instCpCodeMgr.GetIndustryList()
category_info = pd.DataFrame(columns=["category_code", "category_name", "company_nums", "company_list"])
for group in group_list:
    company_list = instCpCodeMgr.GetGroupCodeList(group)
    category_info = category_info.append({"category_code": group, 
                                          "category_name": instCpCodeMgr.GetGroupName(group), 
                                          "company_nums": len(company_list),
                                          "company_list": list(company_list)
                                         }, ignore_index=True)
category_info = category_info[category_info.company_nums != 0]
category_info

,category_code,category_name,company_nums,company_list
1,002,대형(시가총액),100,"[A001040, A000120, A097950, A005830, A078930, ..."
2,003,중형(시가총액),200,"[A027410, A282330, A138930, A079160, A000990, ..."
3,004,소형(시가총액),493,"[A095570, A006840, A001460, A011150, A000590, ..."
4,005,음식료품,47,"[A011150, A011155, A097950, A097955, A023150, ..."
5,006,"섬유,의복",27,"[A001460, A001465, A383220, A007700, A093050, ..."
6,007,"종이,목재",21,"[A002820, A004540, A004545, A014160, A012690, ..."
7,008,화학,122,"[A004840, A089470, A002380, A001390, A025000, ..."
8,009,의약품,52,"[A234080, A001060, A001067, A001065, A302440, ..."
9,010,비금속광물,27,"[A344820, A008870, A005750, A001520, A001527, ..."
10,011,"철강,금속",52,"[A155660, A069730, A016380, A016385, A001940, ..."


#### 음식료품 ~ 제조업까지 카테고리별 종목의 수

In [56]:
five_to_25 = category_info[pd.to_numeric(category_info.category_code) >= 5]
five_to_25 = five_to_25[pd.to_numeric(five_to_25.category_code) <= 25]
five_to_25 = five_to_25.set_index('category_code')
five_to_25

,category_name,company_nums,company_list
category_code,,,
005,음식료품,47,"[A011150, A011155, A097950, A097955, A023150, ..."
006,"섬유,의복",27,"[A001460, A001465, A383220, A007700, A093050, ..."
007,"종이,목재",21,"[A002820, A004540, A004545, A014160, A012690, ..."
008,화학,122,"[A004840, A089470, A002380, A001390, A025000, ..."
009,의약품,52,"[A234080, A001060, A001067, A001065, A302440, ..."
010,비금속광물,27,"[A344820, A008870, A005750, A001520, A001527, ..."
011,"철강,금속",52,"[A155660, A069730, A016380, A016385, A001940, ..."
012,기계,43,"[A082740, A119650, A079550, A009160, A100840, ..."
013,"전기,전자",74,"[A000990, A000995, A092220, A033180, A034220, ..."


#### gropu code 019: 운수창고 (해운) 종목 데이터 가져오기

In [81]:
warehouse = stockitems[stockitems.group_category == 19]
warehouse

,code,name,section,sectionKind,group_category
10,A000120,CJ대한통운,KOSPI,1,19
41,A000650,천일고속,KOSPI,1,19
73,A001140,국보,KOSPI,1,19
136,A002320,한진,KOSPI,1,19
184,A003280,흥아해운,KOSPI,1,19
193,A003490,대한항공,KOSPI,1,19
194,A003495,대한항공우,KOSPI,1,19
226,A004140,동방,KOSPI,1,19
233,A004360,세방,KOSPI,1,19
234,A004365,세방우,KOSPI,1,19


In [84]:
warehouse_stock_data = get_stock_data(warehouse, '19800101','20211231')
warehouse_stock_data

CJ대한통운 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  60
291
19810101~19811231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  59
291
19820101~19821231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  58
296
19830101~19831231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  57
295
19840101~19841231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  56
293
19850101~19851231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  55
294
19860101~19861231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  54
292
19870101~19871231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  53
292
19880101~19881231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  52
293
19890101~19891231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  51
289
19900101~19901231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  50
291
19910101~19911231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  49
292
19920101~19921231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  48
294
19930101~19931231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  47
296
19940101~19941231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은 요청 :  46
296
19950101~19951231의 데이터를 가져옵니다.
A000120 CJ대한통운 남은

19890101~19891231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  45
289
19900101~19901231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  44
291
19910101~19911231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  43
292
19920101~19921231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  42
294
19930101~19931231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  41
296
19940101~19941231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  40
296
19950101~19951231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  39
293
19960101~19961231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  38
293
19970101~19971231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  37
292
19980101~19981231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  36
292
19990101~19991231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  35
249
20000101~20001231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  34
241
20010101~20011231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  33
246
20020101~20021231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  32
244
20030101~20031231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  31
247
20040101~20041231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  30
249
20050101~20051231의 데이터를 가져옵니다.
A002320 한진 남은 요청 :  29
249
20060101~20061

292
19980101~19981231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  30
292
19990101~19991231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  29
249
20000101~20001231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  28
241
20010101~20011231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  27
246
20020101~20021231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  26
244
20030101~20031231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  25
247
20040101~20041231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  24
249
20050101~20051231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  23
249
20060101~20061231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  22
247
20070101~20071231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  21
246
20080101~20081231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  20
248
20090101~20091231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  19
253
20100101~20101231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  18
251
20110101~20111231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  17
248
20120101~20121231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  16
248
20130101~20131231의 데이터를 가져옵니다.
A003495 대한항공우 남은 요청 :  15
247
20140101~20141231의 데

248
20090101~20091231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  13
253
20100101~20101231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  12
251
20110101~20111231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  11
248
20120101~20121231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  10
248
20130101~20131231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  9
247
20140101~20141231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  8
245
20150101~20151231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  7
248
20160101~20161231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  6
246
20170101~20171231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  5
243
20180101~20181231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  4
244
20190101~20191231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  3
246
20200101~20201231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  2
248
20210101~20211231의 데이터를 가져옵니다.
A004365 세방우 남은 요청 :  1
161
한국공항 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A005430 한국공항 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
291
19810101~19811231의 데이터를 가져옵니다.
A005430 한국공항 남은 요청 :  59
291
19820101~19821

대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
246
20170101~20171231의 데이터를 가져옵니다.
A009070 KCTC 남은 요청 :  59
243
20180101~20181231의 데이터를 가져옵니다.
A009070 KCTC 남은 요청 :  58
244
20190101~20191231의 데이터를 가져옵니다.
A009070 KCTC 남은 요청 :  57
246
20200101~20201231의 데이터를 가져옵니다.
A009070 KCTC 남은 요청 :  56
248
20210101~20211231의 데이터를 가져옵니다.
A009070 KCTC 남은 요청 :  55
161
한솔로지스틱스 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  54
0
19810101~19811231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  53
0
19820101~19821231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  52
0
19830101~19831231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  51
0
19840101~19841231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  50
0
19850101~19851231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  49
0
19860101~19861231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  48
0
19870101~19871231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  47
0
19880101~19881231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  46
0
19890101~19891231의 데이터를 가져옵니다.
A009180 한솔로지스틱스 남은 요청 :  45
37
19900101~19901

아시아나항공 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  48
0
19810101~19811231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  47
0
19820101~19821231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  46
0
19830101~19831231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  45
0
19840101~19841231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  44
0
19850101~19851231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  43
0
19860101~19861231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  42
0
19870101~19871231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  41
0
19880101~19881231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  40
0
19890101~19891231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  39
0
19900101~19901231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  38
0
19910101~19911231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  37
0
19920101~19921231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  36
0
19930101~19931231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  35
0
19940101~19941231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  34
0
19950101~19951231의 데이터를 가져옵니다.
A020560 아시아나항공 남은 요청 :  33
0
19960101~19961231의

A084670 동양고속 남은 요청 :  30
0
19930101~19931231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  29
0
19940101~19941231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  28
0
19950101~19951231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  27
0
19960101~19961231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  26
0
19970101~19971231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  25
0
19980101~19981231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  24
0
19990101~19991231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  23
0
20000101~20001231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  22
0
20010101~20011231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  21
0
20020101~20021231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  20
0
20030101~20031231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  19
0
20040101~20041231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  18
0
20050101~20051231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  17
111
20060101~20061231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  16
247
20070101~20071231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  15
246
20080101~20081231의 데이터를 가져옵니다.
A084670 동양고속 남은 요청 :  14
248
20090101~20091231의 데이터를 가져옵니다.
A08467

20020101~20021231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  14
0
20030101~20031231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  13
0
20040101~20041231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  12
0
20050101~20051231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  11
0
20060101~20061231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  10
0
20070101~20071231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  9
0
20080101~20081231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  8
0
20090101~20091231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  7
0
20100101~20101231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  6
0
20110101~20111231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  5
0
20120101~20121231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  4
0
20130101~20131231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  3
0
20140101~20141231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  2
0
20150101~20151231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  1
0
20160101~20161231의 데이터를 가져옵니다.
A091810 티웨이항공 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
0
20170101~20171231의 데이터를 

A18064K 한진칼우 남은 요청 :  55
248
20160101~20161231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  54
246
20170101~20171231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  53
243
20180101~20181231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  52
244
20190101~20191231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  51
246
20200101~20201231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  50
248
20210101~20211231의 데이터를 가져옵니다.
A18064K 한진칼우 남은 요청 :  49
161
진에어 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  48
0
19810101~19811231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  47
0
19820101~19821231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  46
0
19830101~19831231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  45
0
19840101~19841231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  44
0
19850101~19851231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  43
0
19860101~19861231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  42
0
19870101~19871231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  41
0
19880101~19881231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  40
0
19890101~19891231의 데이터를 가져옵니다.
A272450 진에어 남은 요청 :  39
0
19900101~19901231의 데이터

,code,name,section,date,open,high,low,close,vol,value,n_stock,agg_price,foreign_vol,foreign_rate,agency_buy,agency_netbuy
0,A000120,CJ대한통운,KOSPI,19800104,7219,7219,7219,7219,6,0,0,0,0,0.00,0,0
1,A000120,CJ대한통운,KOSPI,19800105,7191,7191,7191,7191,0,0,0,0,0,0.00,0,0
2,A000120,CJ대한통운,KOSPI,19800107,6941,7080,6941,7080,95,0,0,0,0,0.00,0,0
3,A000120,CJ대한통운,KOSPI,19800108,7108,7358,7108,7219,283,0,0,0,0,0.00,0,0
4,A000120,CJ대한통운,KOSPI,19800109,7219,7219,7080,7080,314,0,0,0,0,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,A298690,에어부산,KOSPI,20210818,2670,2825,2670,2770,275628,765000000,82070000,227333000000,351996,0.43,48681,-4043061
653,A298690,에어부산,KOSPI,20210819,2770,2790,2640,2650,236030,636000000,82070000,217485000000,331687,0.40,-22808,-4065869
654,A298690,에어부산,KOSPI,20210820,2645,2700,2520,2560,320827,823000000,82070000,210099000000,326241,0.40,-19472,-4085341
655,A298690,에어부산,KOSPI,20210823,2535,2625,2535,2610,186882,484000000,82070000,214202000000,351952,0.43,12313,-4073028


In [92]:
warehouse_stock_data.to_csv("./data/warehouse-1980-2021.csv", index=False)
warehouse_stock_data = pd.read_csv("./data/warehouse-1980-2021.csv")
warehouse_stock_data

,code,name,section,date,open,high,low,close,vol,value,n_stock,agg_price,foreign_vol,foreign_rate,agency_buy,agency_netbuy
0,A000120,CJ대한통운,KOSPI,19800104,7219,7219,7219,7219,6,0,0,0,0,0.00,0,0
1,A000120,CJ대한통운,KOSPI,19800105,7191,7191,7191,7191,0,0,0,0,0,0.00,0,0
2,A000120,CJ대한통운,KOSPI,19800107,6941,7080,6941,7080,95,0,0,0,0,0.00,0,0
3,A000120,CJ대한통운,KOSPI,19800108,7108,7358,7108,7219,283,0,0,0,0,0.00,0,0
4,A000120,CJ대한통운,KOSPI,19800109,7219,7219,7080,7080,314,0,0,0,0,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183512,A298690,에어부산,KOSPI,20210818,2670,2825,2670,2770,275628,765000000,82070000,227333000000,351996,0.43,48681,-4043061
183513,A298690,에어부산,KOSPI,20210819,2770,2790,2640,2650,236030,636000000,82070000,217485000000,331687,0.40,-22808,-4065869
183514,A298690,에어부산,KOSPI,20210820,2645,2700,2520,2560,320827,823000000,82070000,210099000000,326241,0.40,-19472,-4085341
183515,A298690,에어부산,KOSPI,20210823,2535,2625,2535,2610,186882,484000000,82070000,214202000000,351952,0.43,12313,-4073028
